In [1]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets
!pip install -q scipy



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python -m pip install --upgrade pip


In [3]:
model_id = 'bigscience/bloom-7b1'
#model_id = 'bigscience/mt0-large'
#model_id = 'tiiuae/falcon-7b'

In [4]:
!pip install einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.2 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python -m pip install --upgrade pip


In [5]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForCausalLM, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
#model_id = 'merged'
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map='auto', trust_remote_code=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [7]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)

In [ ]:
from datasets import load_dataset

data = load_dataset("thanhnew2001/alpaca_vn")

data = data.map(
    lambda samples: {
        "input_text": ' ### Human: ' + samples["question"] + ' ### Assisstant: ' + samples["answer"] + ' ### '
    }
, remove_columns= ['question', 'answer'])

data = data.map(lambda examples: tokenizer(examples["input_text"]), batched=True)


#print(data['train'])
#data['train'][0:5]

#data = data.map(lambda x: tokenizer(f"{x['question']}"), batched=True)

In [ ]:
import transformers

# needed for gpt-neo-x tokenizer
tokenizer.pad_token = tokenizer.eos_token

trainer = transformers.Trainer(
    model=model,
    train_dataset=data["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=2,
        max_steps=10,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=100,
        output_dir="outputs",
        optim="paged_adamw_8bit"
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

In [ ]:
trainer.save_model('adapter-model')


In [10]:
from peft import PeftModel    

base_model = AutoModelForCausalLM.from_pretrained(
        model_id,
        return_dict=True,
        torch_dtype=torch.float16,
        trust_remote_code=True
)

model = PeftModel.from_pretrained(base_model, 'adapter-model').to('cuda')
print(f"Running merge_and_unload")
model = model.merge_and_unload() 
tokenizer = AutoTokenizer.from_pretrained(model_id)

model.save_pretrained("merged2")
tokenizer.save_pretrained("merged2")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Running merge_and_unload


('merged2/tokenizer_config.json',
 'merged2/special_tokens_map.json',
 'merged2/tokenizer.json')

In [ ]:

model_id ='merged2'
model_double = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto", trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [12]:
text = "Làm thế nào vượt qua nghịch cảnh?"
inputs = tokenizer(text, return_tensors="pt", return_token_type_ids=False).to("cuda:0")
outputs = model.generate(**inputs, max_new_tokens=200, pad_token_id=50256)
print(outputs)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

tensor([[ 52676,   4822,   5541,  32670,   6131, 101289,  11926,     34, 104300,
          14674,   9449,   1120,  20596,   4959,  17382,   3377,   8484,  10960,
           1123,   4959,  17382,   3377,   3067,   4850,   6205,   1353,   3339,
           3173,     17,  76069,  11075,   6757,   6205,   1353,   3339,   3173,
           2001,   1123,   6205,   1353,   3339,  26772,   2001,  11283,   6783,
           5301,   4850,   6205,   7350,     17,  76069,  11075,   6757,   6205,
           1353,   3339,  32670,   6131, 101289,  11926,   1123,  26772,   2001,
           3067,   5301,  12788,  17837,     17,  76069,  11075,   6757,   6205,
           1353,   3339,   3173,   2001,   1123,   6205,   1353,   3339,  26772,
           2001,  11283,   6783,   5301,   4850,   6205,   7350,     17,  76069,
          11075,   6757,   6205,   1353,   3339,  32670,   6131, 101289,  11926,
           1123,  26772,   2001,   3067,   5301,  12788,  17837,     17,  76069,
          11075,   6757,   6